In [115]:
# pip install --upgrade snowflake-connector-python
# pip install --upgrade snowflake-sqlalchemy



In [29]:
import os
import snowflake.connector
import glob
import pandas as pd

# Question 1:

1. **Extract** (you can manually extract all the data files in the provided `data.zip` file) and load the 41 comma-delimited purchases data files and form a single table of purchases data (you should load the 41 CSV files into a Snowflake stage and then move the data from the stage to a Snowflake table):
   
   a. Preferably follow these guidelines when staging the files (this staging approach does not make sense for our data as the files are small, but it is good practice if you have more data and if the data is loaded over time).
   
   b. Use Python to automate the `PUT` process, e.g., use `glob` to iterate through and `PUT` all purchases files automatically.
   
   c. You can examine the data in the stage using regular SQL statements, but where columns are referred to using the positional number of the column preceded by `$`, e.g., `SELECT $.1, $.3 FROM...` selects the first and second column in the staged data: [Snowflake Staging Documentation](https://docs.snowflake.com/user-guide/querying-stage).
   
   d. `COPY INTO` is generally preferred over `INSERT INTO` (this applies to the entire project).
   
   e. To the extent possible, perform transformations such as selecting columns and setting data types during the `COPY INTO` process. There are a number of columns that are not needed in the project. You can exclude columns that appear to not have any useful information (e.g., the same value on each row, only null values, etc.). You can also exclude columns that you do not need for the project (look through the instructions and try to determine which columns will be needed. If you realize later that you excluded columns that you need, then simply come back to this code and change it to include the additional column(s) that are causing errors).

   f.	If you have multiple steps in your code that are needed for moving data from source into the final table and the final output is not as expected (e.g., you can count the number of rows in the raw data and then verify that you have the same number of rows in the final Snowflake table), then try to troubleshoot your code by verifying which step in your process does not produce the expect results. This will for example require examining staged data. Start at the beginning when doing this. When you have located the step that does not produce the expected then start troubleshooting this step in more detail.

In [27]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user='wbkennedy',
    password='Brandon92!',
    account='nulgoll-hsb06466',
    warehouse='final_project_warehouse',
    database='final_project_db',
    schema='final_project_schema'
)

cs = conn.cursor()

In [3]:
#create a virtual warehouse (virtual warehouses contain the compute resources that are required to perform queries and DML operations with Snowflake)
cs.execute("CREATE WAREHOUSE IF NOT EXISTS final_project_warehouse")

# create a database
cs.execute("CREATE DATABASE IF NOT EXISTS final_project_db")

# create schema
cs.execute("CREATE SCHEMA IF NOT EXISTS final_project_schema")

In [6]:
# Create Snowflake stage to hold the files
stage_name = "purchases_stage"
cs.execute(f"CREATE OR REPLACE STAGE {stage_name}")

# Folder containing the CSV files (your provided path)
csv_folder_path = '/home/jovyan/MGTA_464/SQLETLSnowflake/CaseData/Data/Monthly PO Data'

# Use glob to iterate through all CSV files in the directory and upload them to the stage
for file_path in glob.glob(os.path.join(csv_folder_path, "*.csv")):
    file_name = os.path.basename(file_path)
    
    # Make sure to escape special characters or use quotes for file paths
    file_path_escaped = file_path.replace(" ", "\\ ")
    
    # Upload the file to the Snowflake stage
    put_query = f"PUT 'file://{file_path_escaped}' @{stage_name}/{file_name}"
    cs.execute(put_query)
    print(f"Uploaded {file_name} to Snowflake stage")

# Create a target table in Snowflake with proper data types
cs.execute("""
    CREATE OR REPLACE TABLE purchases_data (
        PurchaseOrderID INTEGER,
        SupplierID INTEGER,
        OrderDate DATE,
        DeliveryMethodID INTEGER,
        ContactPersonID INTEGER,
        ExpectedDeliveryDate DATE,
        SupplierReference STRING,
        IsOrderFinalized BOOLEAN,
        Comments STRING,
        InternalComments STRING,
        LastEditedBy INTEGER,
        LastEditedWhen TIMESTAMP,
        PurchaseOrderLineID INTEGER,
        StockItemID INTEGER,
        OrderedOuters INTEGER,
        Description STRING,
        ReceivedOuters INTEGER,
        PackageTypeID INTEGER,
        ExpectedUnitPricePerOuter FLOAT,
        LastReceiptDate DATE,
        IsOrderLineFinalized BOOLEAN,
        Right_LastEditedBy INTEGER,
        Right_LastEditedWhen TIMESTAMP
    );
""")
print("Created table purchases_data")

# Use COPY INTO to move data from the stage to the Snowflake table, with correct formatting and error handling
copy_into_query = f"""
COPY INTO purchases_data
FROM @{stage_name}
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    SKIP_HEADER = 1,
    DATE_FORMAT = 'MM/DD/YYYY',
    TIMESTAMP_FORMAT = 'MM/DD/YYYY HH24:MI'
)
ON_ERROR = 'CONTINUE';
"""
cs.execute(copy_into_query)
print("Data copied from stage to purchases_data table")

# Remove the files from the stage after loading them to the table
cs.execute(f"REMOVE @{stage_name}")
print(f"Removed files from stage {stage_name}")


Uploaded 2020-2.csv to Snowflake stage
Uploaded 2021-3.csv to Snowflake stage
Uploaded 2020-5.csv to Snowflake stage
Uploaded 2021-11.csv to Snowflake stage
Uploaded 2022-1.csv to Snowflake stage
Uploaded 2019-8.csv to Snowflake stage
Uploaded 2021-7.csv to Snowflake stage
Uploaded 2021-10.csv to Snowflake stage
Uploaded 2019-6.csv to Snowflake stage
Uploaded 2021-8.csv to Snowflake stage
Uploaded 2020-10.csv to Snowflake stage
Uploaded 2020-8.csv to Snowflake stage
Uploaded 2021-1.csv to Snowflake stage
Uploaded 2019-4.csv to Snowflake stage
Uploaded 2019-12.csv to Snowflake stage
Uploaded 2022-4.csv to Snowflake stage
Uploaded 2020-11.csv to Snowflake stage
Uploaded 2019-10.csv to Snowflake stage
Uploaded 2020-12.csv to Snowflake stage
Uploaded 2020-3.csv to Snowflake stage
Uploaded 2020-1.csv to Snowflake stage
Uploaded 2019-2.csv to Snowflake stage
Uploaded 2019-11.csv to Snowflake stage
Uploaded 2021-2.csv to Snowflake stage
Uploaded 2019-3.csv to Snowflake stage
Uploaded 2019-9.c

# Question 2:

2. **Create a calculated field** that shows purchase order totals, i.e., for each order, sum the line-item amounts (defined as `ReceivedOuters * ExpectedUnitPricePerOuter`), and name this field `POAmount`.

In [7]:
# Add the POAmount column
cs.execute("""
    ALTER TABLE purchases_data
    ADD COLUMN POAmount FLOAT;
""")
print("Added POAmount column")

# Update the table to calculate POAmount
cs.execute("""
    UPDATE purchases_data
    SET POAmount = ReceivedOuters * ExpectedUnitPricePerOuter;
""")
print("Updated POAmount values")

Added POAmount column
Updated POAmount values


# Question 3:

3. **Load the supplier invoice XML data** (you will again first stage the data and then move it into a table):

   a. Shred the data into a table (preferably in the `COPY INTO` process) where each row corresponds to a single invoice.
   
   b. Make sure to examine the structure of the XML file and also try different functions such as `GETXML`, `GET`, `PARSE_XML`, `FLATTEN`, etc.
   
   c. When building your query to shred the data, try to keep it as simple as possible at first and only attempt to extract a single element or only try a single SQL clause or function to see what it produces.

In [8]:
# Create a stage for uploading XML data
cs.execute("CREATE OR REPLACE STAGE xml_stage")
print("Stage 'xml_stage' created or exists.")

# Create an XML file format in Snowflake
cs.execute("""
    CREATE OR REPLACE FILE FORMAT xml_file_format 
    TYPE = 'XML' 
    STRIP_OUTER_ELEMENT = TRUE
""")
print("XML file format 'xml_file_format' created.")

# Upload the XML file to the Snowflake stage
xml_file_path = "/home/jovyan/MGTA_464/SQLETLSnowflake/CaseData/Data/Supplier Transactions XML.xml"
put_command = f"PUT 'file://{xml_file_path}' @xml_stage auto_compress=true"
cs.execute(put_command)
print("XML file uploaded to stage.")

# Create a table to store the raw XML data
cs.execute("""
    CREATE OR REPLACE TABLE RawXMLData (
        xml_column VARIANT
    )
""")
print("Table 'RawXMLData' created or replaced successfully.")

# Load the raw XML data into the table
try:
    cs.execute("""
        COPY INTO RawXMLData 
        FROM @xml_stage 
        FILE_FORMAT = (FORMAT_NAME = 'xml_file_format') 
        ON_ERROR = 'CONTINUE'
    """)
    print("XML data loaded into 'RawXMLData' successfully.")
except Exception as e:
    print(f"Error loading XML data: {e}")

# Create the SupplierInvoices table
cs.execute("""
    CREATE OR REPLACE TABLE SupplierInvoices (
        SupplierTransactionID INTEGER,
        SupplierID INTEGER,
        TransactionTypeID INTEGER,
        PurchaseOrderID INTEGER,
        PaymentMethodID INTEGER,
        SupplierInvoiceNumber STRING,
        TransactionDate DATE,
        AmountExcludingTax FLOAT,
        TaxAmount FLOAT,
        TransactionAmount FLOAT,
        OutstandingBalance FLOAT,
        FinalizationDate DATE,
        IsFinalized BOOLEAN,
        LastEditedBy INTEGER,
        LastEditedWhen TIMESTAMP_NTZ
    )
""")
print("Table 'SupplierInvoices' created or replaced successfully.")

# Transform and load data from RawXMLData to SupplierInvoices using XMLGET()
try:
    cs.execute("""
        INSERT INTO SupplierInvoices
        SELECT 
            TRY_TO_NUMBER(XMLGET(xml_column, 'SupplierTransactionID'):"$"::STRING) AS SupplierTransactionID,
            TRY_TO_NUMBER(XMLGET(xml_column, 'SupplierID'):"$"::STRING) AS SupplierID,
            TRY_TO_NUMBER(XMLGET(xml_column, 'TransactionTypeID'):"$"::STRING) AS TransactionTypeID,
            TRY_TO_NUMBER(XMLGET(xml_column, 'PurchaseOrderID'):"$"::STRING) AS PurchaseOrderID,
            TRY_TO_NUMBER(XMLGET(xml_column, 'PaymentMethodID'):"$"::STRING) AS PaymentMethodID,
            XMLGET(xml_column, 'SupplierInvoiceNumber'):"$"::STRING AS SupplierInvoiceNumber,
            TRY_TO_DATE(XMLGET(xml_column, 'TransactionDate'):"$"::STRING, 'YYYY-MM-DD') AS TransactionDate,
            TRY_TO_NUMBER(XMLGET(xml_column, 'AmountExcludingTax'):"$"::STRING) AS AmountExcludingTax,
            TRY_TO_NUMBER(XMLGET(xml_column, 'TaxAmount'):"$"::STRING) AS TaxAmount,
            TRY_TO_NUMBER(XMLGET(xml_column, 'TransactionAmount'):"$"::STRING) AS TransactionAmount,
            TRY_TO_NUMBER(XMLGET(xml_column, 'OutstandingBalance'):"$"::STRING) AS OutstandingBalance,
            TRY_TO_DATE(XMLGET(xml_column, 'FinalizationDate'):"$"::STRING, 'YYYY-MM-DD') AS FinalizationDate,
            TRY_TO_BOOLEAN(XMLGET(xml_column, 'IsFinalized'):"$"::STRING) AS IsFinalized,
            TRY_TO_NUMBER(XMLGET(xml_column, 'LastEditedBy'):"$"::STRING) AS LastEditedBy,
            TRY_TO_TIMESTAMP_NTZ(XMLGET(xml_column, 'LastEditedWhen'):"$"::STRING, 'YYYY-MM-DD HH24:MI:SS.FF') AS LastEditedWhen
        FROM RawXMLData
    """)
    print("Transformed and loaded XML data into 'SupplierInvoices' successfully using XMLGET().")
except Exception as e:
    print(f"Error transforming and loading XML data using XMLGET(): {e}")

Stage 'xml_stage' created or exists.
XML file format 'xml_file_format' created.
XML file uploaded to stage.
Table 'RawXMLData' created or replaced successfully.
XML data loaded into 'RawXMLData' successfully.
Table 'SupplierInvoices' created or replaced successfully.
Transformed and loaded XML data into 'SupplierInvoices' successfully using XMLGET().


# Question 4:

4. **Join the purchases data from step 2** and the supplier invoices data from step 3 (only include matching rows).

# Question 5:

5. Using the joined data from step 4, create a calculated field that shows the difference between `AmountExcludingTax` and `POAmount`, name this field `invoiced_vs_quoted`, and save the result as a materialized view named `purchase_orders_and_invoices`:
   
   a. If your version of Snowflake does not support materialized views, then create a table instead using the join.

In [9]:
cs.execute("""
    CREATE OR REPLACE TABLE purchase_orders_and_invoices AS
    SELECT 
    
        (SI.AmountExcludingTax - PD.POAmount) AS invoiced_vs_quoted,
    FROM PURCHASES_DATA PD
    INNER JOIN SupplierInvoices SI
    ON PD.PurchaseOrderID = SI.PurchaseOrderID
""")
conn.commit()
print("Table 'purchase_orders_and_invoices' created.")

Table 'purchase_orders_and_invoices' created.


# Question 6:

6. Manually open the `supplier_case` SQL script (in the SQL editor that you have used in class previously, e.g., VS Code) and run the code to create the `supplier_case` table (you can create the table in `WestCoastImporters` or any other database). Then extract the `supplier_case` data from the Postgres table you just created (do not import the data into Python) by using Python to move the data from Postgres directly to your local drive and then directly into a Snowflake stage.

   a. Consider creating a Python function that can take a CSV file path as input and then generate field definitions (field names and datatypes based on the header and data types in the file) that can then be used in a `CREATE TABLE` statement.
   
   b. You need to use `psycopg2` or a similar Python library to connect to the Postgres database within Python, issue a command to Postgres to have Postgres save the `supplier_case` data to a file, and then use `cs.execute` to move the file to an internal Snowflake stage and eventually into a table.

In [10]:
import psycopg2
import csv

def export_postgres_to_csv(db_conn_str, query, output_file):
    conn = psycopg2.connect(db_conn_str)
    cursor = conn.cursor()
    
    with open(output_file, 'w') as f:
        cursor.copy_expert(f"COPY ({query}) TO STDOUT WITH CSV HEADER", f)
    
    cursor.close()
    conn.close()
    print(f"Data exported to {output_file}")

# Connection string to your Postgres database
postgres_conn_str = "dbname='WestCoastImporters' user='jovyan' host='127.0.0.1' port='8765' password='postgres'"

# SQL query to extract the data from the table
query = "SELECT * FROM supplier_case"

# Export data to CSV
export_postgres_to_csv(postgres_conn_str, query, "supplier_case.csv")

# Upload the CSV file to the Snowflake stage
stage_name = "supplier_stage"
csv_file = "supplier_case.csv"

cs.execute(f"PUT file://{csv_file} @{stage_name}")
print(f"Uploaded {csv_file} to Snowflake stage")


Data exported to supplier_case.csv
Uploaded supplier_case.csv to Snowflake stage


In [11]:
# Create the table before loading data
cs.execute("""
    CREATE OR REPLACE TABLE supplier_case (
        supplierid STRING,
        suppliername STRING,
        suppliercategoryid STRING,
        primarycontactpersonid STRING,
        alternatecontactpersonid STRING,
        deliverymethodid STRING,
        postalcityid STRING,
        supplierreference STRING,
        bankaccountname STRING,
        bankaccountbranch STRING,
        bankaccountcode STRING,
        bankaccountnumber STRING,
        bankinternationalcode STRING,
        paymentdays INT,
        internalcomments STRING,
        phonenumber STRING,
        faxnumber STRING,
        websiteurl STRING,
        deliveryaddressline1 STRING,
        deliveryaddressline2 STRING,
        deliverypostalcode STRING,
        deliverylocation STRING,
        postaladdressline1 STRING,
        postaladdressline2 STRING,
        postalpostalcode STRING,
        lasteditedby STRING,
        validfrom STRING,
        validto STRING
    )
""")

print("Created supplier_case table in Snowflake")

# Define a file format with proper handling for enclosed fields
cs.execute("""
    CREATE OR REPLACE FILE FORMAT my_csv_format
    TYPE = 'CSV'
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    SKIP_HEADER = 1
    NULL_IF = ('');
""")

# Copy data from the Snowflake stage to the table using the custom file format
cs.execute(f"""
    COPY INTO supplier_case
    FROM @{stage_name}/{csv_file.split('/')[-1]}
    FILE_FORMAT = my_csv_format;
""")
print("Data loaded into the supplier_case table")

Created supplier_case table in Snowflake
Data loaded into the supplier_case table


# Question 7:

7. **Connect manually to NOAA data using Marketplace**. From inside Snowflake Marketplace (from the home screen, click **Data Products**) search for **NOAA** and then select **Weather & Environment from Cybersyn** (click **Get**). The name of the datasets that you will be using can be accessed in SQL queries running on Snowflake using `cybersyn.noaa_weather_metrics_timeseries` and `cybersyn.noaa_weather_station_index` (`NOAA_WEATHER_METRICS_ATTRIBUTES` additionally contains data definitions that might be helpful). Using this data, extract weather data for each unique zip code in the `supplier_case` table (suppliers can have the same zip code, but you only need to extract weather data for each zip code once).

   a. While the weather station data contain zip codes, we will pretend that this table does not have this information and instead use latitude and longitude information to determine which weather station to use for each zip code. The approach used in [this article](https://towardsdatascience.com/noaa-weather-data-in-snowflake-free-20e90ee916ed) can be helpful (note that this is based on a different dataset, but the idea of using latitude and longitude is the same) for finding weather stations closest to each zip code (only use one weather station per zip code). For this to work, you need to find a data file with zip code – geo-location mappings, e.g., from the US Census (the data zip folder on Canvas contains a ZCTA file with this information; in this file `GEOID` is the five-digit ZIP Code, `INTPTLAT` is Latitude, and `INTPTLONG` is Longitude).

   b. Create a **materialized view** named `supplier_zip_code_weather` that contains the unique zip codes (`PostalPostalCode`) from the supplier data, date, and daily high temperatures. The view should have three columns: `zip code`, `date`, and `high temperature`, and one row per day and unique supplier zip code. You will not have temperature data for all the suppliers. This is fine.

In [55]:
# Specify 'GEOID' as a string to preserve leading zeros when loading the file
df_zip_geo = pd.read_csv('/home/jovyan/MGTA_464/SQLETLSnowflake/CaseData/Data/2021_Gaz_zcta_national.txt', delimiter='\t', dtype={'GEOID': str})

# Check the first few rows of the DataFrame to ensure leading zeros are preserved
print(df_zip_geo.head())


   GEOID      ALAND   AWATER  ALAND_SQMI  AWATER_SQMI   INTPTLAT  \
0  00601  166847909   799292      64.420        0.309  18.180555   
1  00602   78546713  4428428      30.327        1.710  18.361945   
2  00603   88957333  6276536      34.347        2.423  18.458497   
3  00606  114825382    12487      44.334        0.005  18.158327   
4  00610   96129350  4310530      37.116        1.664  18.294032   

   INTPTLONG                                                                                                                                    
0                                         -66.749961                                                                                            
1                                         -67.175597                                                                                            
2                                         -67.123906                                                                                            
3          

In [56]:
# Save the DataFrame as a CSV file
output_csv_path = '/home/jovyan/MGTA_464/SQLETLSnowflake/CaseData/Data/2021_Gaz_zcta_national.csv'
df_zip_geo.to_csv(output_csv_path, index=False)

print(f"DataFrame successfully written to {output_csv_path}")

DataFrame successfully written to /home/jovyan/MGTA_464/SQLETLSnowflake/CaseData/Data/2021_Gaz_zcta_national.csv


In [57]:
# Step 1: Create the zip_geo_mapping table with all columns
cs.execute("""
    CREATE OR REPLACE TABLE zip_geo_mapping (
        GEOID STRING,
        ALAND FLOAT,
        AWATER FLOAT,
        ALAND_SQMI FLOAT,
        AWATER_SQMI FLOAT,
        INTPTLAT FLOAT,
        INTPTLONG FLOAT
    );
""")
print("Table zip_geo_mapping with all columns created successfully.")

# Step 2: Create a stage to upload the CSV file
stage_name = "zip_geo_stage"
cs.execute(f"CREATE OR REPLACE STAGE {stage_name}")
print(f"Stage {stage_name} created successfully.")

# Step 3: Upload the CSV file to the Snowflake stage
csv_file_path = output_csv_path

# Perform the upload to Snowflake's stage
put_query = f"PUT file://{csv_file_path} @{stage_name}"
cs.execute(put_query)
print(f"Uploaded {os.path.basename(csv_file_path)} to Snowflake stage")

# Step 4: Copy the data from the stage into the zip_geo_mapping table
copy_into_query = f"""
COPY INTO zip_geo_mapping
FROM @{stage_name}
FILE_FORMAT = (
    TYPE = 'CSV',
    FIELD_OPTIONALLY_ENCLOSED_BY = '"',
    SKIP_HEADER = 1
)
ON_ERROR = 'CONTINUE';
"""
cs.execute(copy_into_query)
print("Data copied from stage to zip_geo_mapping table")

# Step 5: Verify the data loaded into the table
cs.execute("SELECT * FROM zip_geo_mapping LIMIT 10;")
rows = cs.fetchall()
print("Sample data from zip_geo_mapping:")
for row in rows:
    print(row)

# Step 6: Clean up the stage (optional)
cs.execute(f"REMOVE @{stage_name}")
print(f"Removed files from stage {stage_name}")


Table zip_geo_mapping with all columns created successfully.
Stage zip_geo_stage created successfully.
Uploaded 2021_Gaz_zcta_national.csv to Snowflake stage
Data copied from stage to zip_geo_mapping table
Sample data from zip_geo_mapping:
('00601', 166847909.0, 799292.0, 64.42, 0.309, 18.180555, -66.749961)
('00602', 78546713.0, 4428428.0, 30.327, 1.71, 18.361945, -67.175597)
('00603', 88957333.0, 6276536.0, 34.347, 2.423, 18.458497, -67.123906)
('00606', 114825382.0, 12487.0, 44.334, 0.005, 18.158327, -66.932928)
('00610', 96129350.0, 4310530.0, 37.116, 1.664, 18.294032, -67.127156)
('00611', 27570859.0, 3631.0, 10.645, 0.001, 18.276316, -66.807165)
('00612', 197202754.0, 15258391.0, 76.14, 5.891, 18.416727, -66.70009)
('00616', 28189755.0, 143499.0, 10.884, 0.055, 18.420412, -66.671979)
('00617', 47538645.0, 1692937.0, 18.355, 0.654, 18.446889, -66.561154)
('00622', 80724808.0, 19583150.0, 31.168, 7.561, 17.988103, -67.160357)
Removed files from stage zip_geo_stage


In [20]:
# try:
#     # Use the correct database name with double underscores
#     cs.execute('USE DATABASE "WEATHER__ENVIRONMENT";')
    
#     # Check if CYBERSYN schema exists, if so use it
#     cs.execute('USE SCHEMA "CYBERSYN";')
    
#     print("Switched to WEATHER__ENVIRONMENT database and CYBERSYN schema.")
    
# except snowflake.connector.errors.ProgrammingError as e:
#     print(f"Error switching database/schema: {e}")
    
#     # Debugging: List available schemas in the WEATHER__ENVIRONMENT database
#     cs.execute('SHOW SCHEMAS IN DATABASE "WEATHER__ENVIRONMENT";')
#     schemas = cs.fetchall()
#     print("Available schemas in WEATHER__ENVIRONMENT:")
#     for schema in schemas:
#         print(schema[1])  # Second column is the schema name

Switched to WEATHER__ENVIRONMENT database and CYBERSYN schema.


In [23]:
# # Check the structure of the NOAA_WEATHER_METRICS_TIMESERIES view
# cs.execute('DESCRIBE VIEW NOAA_WEATHER_METRICS_TIMESERIES;')
# metrics_structure = cs.fetchall()
# print("NOAA_WEATHER_METRICS_TIMESERIES structure:")
# for col in metrics_structure:
#     print(col)

NOAA_WEATHER_METRICS_TIMESERIES structure:
('NOAA_WEATHER_STATION_ID', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Unique identifier for land surface weather stations in the Global Historical Climatology Network daily (GHCNd), comprised of the FIPS country code (first 2 characters), a network code (third character), and the actual station ID (the remaining 8 characters).', None, None)
('VARIABLE', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'A unique identifier for a variable joinable to the timeseries table.', None, None)
('VARIABLE_NAME', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Human-readable name for the variable.', None, None)
('DATE', 'DATE', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Date associated with the value', None, None)
('DATETIME', 'TIMESTAMP_NTZ(9)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Datetime associated with the value', None, None)
('VALUE', 'NUMBER(38,6)', 'COLUMN', 'Y', None, 'N', 'N', None, 

In [22]:
# # Check the structure of the NOAA_WEATHER_STATION_INDEX view
# cs.execute('DESCRIBE VIEW NOAA_WEATHER_STATION_INDEX;')
# station_structure = cs.fetchall()
# print("NOAA_WEATHER_STATION_INDEX structure:")
# for col in station_structure:
#     print(col)

NOAA_WEATHER_STATION_INDEX structure:
('NOAA_WEATHER_STATION_ID', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Unique identifier for land surface weather stations in the Global Historical Climatology Network daily (GHCNd), comprised of the FIPS country code (first 2 characters), a network code (third character), and the actual station ID (the remaining 8 characters).', None, None)
('NOAA_WEATHER_STATION_NAME', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Full name of the weather station.', None, None)
('COUNTRY_GEO_ID', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, "Cybersyn's unique identifier for a country, joinable to other datasets.", None, None)
('COUNTRY_NAME', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, 'Full name of the country.', None, None)
('STATE_GEO_ID', 'VARCHAR(16777216)', 'COLUMN', 'Y', None, 'N', 'N', None, None, "Cybersyn's unique identifier for a state, joinable to other datasets.", None, N

In [44]:
# # List all tables in the current schema to verify the existence of zip_geo_mapping
# cs.execute("SHOW TABLES;")
# tables = cs.fetchall()

# # Print the available tables
# print("Available tables in the schema:")
# for table in tables:
#     print(table[1])  # Second column is the table name


Available tables in the schema:
PURCHASES_DATA
PURCHASE_ORDERS_AND_INVOICES
RAWXMLDATA
SUPPLIERINVOICES
SUPPLIER_CASE
SUPPLIER_TRANSACTIONS
SUPPLIER_TRANSACTIONS_RAW
ZIP_GEO_MAPPING


In [46]:
# Corrected query to map ZIP codes to the closest weather station
haversine_query = """
WITH closest_stations AS (
    SELECT
        z.GEOID AS zip_code,
        z.INTPTLAT AS zip_lat,
        z.INTPTLONG AS zip_lon,
        s.NOAA_WEATHER_STATION_ID,
        s.LATITUDE AS station_lat,
        s.LONGITUDE AS station_lon,
        -- Haversine formula to calculate distance
        (6371 * ACOS(
            COS(RADIANS(z.INTPTLAT)) * COS(RADIANS(s.LATITUDE)) *
            COS(RADIANS(s.LONGITUDE) - RADIANS(z.INTPTLONG)) +
            SIN(RADIANS(z.INTPTLAT)) * SIN(RADIANS(s.LATITUDE))
        )) AS distance_km,
        ROW_NUMBER() OVER (PARTITION BY z.GEOID ORDER BY 
            (6371 * ACOS(
                COS(RADIANS(z.INTPTLAT)) * COS(RADIANS(s.LATITUDE)) *
                COS(RADIANS(s.LONGITUDE) - RADIANS(z.INTPTLONG)) +
                SIN(RADIANS(z.INTPTLAT)) * SIN(RADIANS(s.LATITUDE))
            )) ASC) AS rank
    FROM FINAL_PROJECT_DB.FINAL_PROJECT_SCHEMA.ZIP_GEO_MAPPING z
    JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX s
    ON TRUE
)
-- Select the closest station for each ZIP code
SELECT
    zip_code,
    NOAA_WEATHER_STATION_ID,
    distance_km
FROM closest_stations
WHERE rank = 1;
"""

cs.execute(haversine_query)
closest_station_data = cs.fetchall()


In [47]:
# Convert the result to a Pandas DataFrame
df_closest_station = pd.DataFrame(closest_station_data, columns=['zip_code', 'NOAA_WEATHER_STATION_ID', 'closest_station_distance'])
print(df_closest_station.head())

Empty DataFrame
Columns: [zip_code, NOAA_WEATHER_STATION_ID, closest_station_distance]
Index: []


In [ ]:
# Query to retrieve daily high temperatures for each ZIP code
weather_query = """
WITH closest_stations AS (
    -- Get the closest station for each ZIP code
    SELECT zip_code, NOAA_WEATHER_STATION_ID
    FROM FINAL_PROJECT_DB.FINAL_PROJECT_SCHEMA.ZIP_GEO_MAPPING z
    JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX s
    ON TRUE
    QUALIFY ROW_NUMBER() OVER (PARTITION BY z.GEOID ORDER BY 
    (6371 * ACOS(
        COS(RADIANS(z.INTPTLAT)) * COS(RADIANS(s.LATITUDE)) *
        COS(RADIANS(s.LONGITUDE) - RADIANS(z.INTPTLONG)) +
        SIN(RADIANS(z.INTPTLAT)) * SIN(RADIANS(s.LATITUDE))
    )) ASC) = 1
)
SELECT
    cs.zip_code,
    wm.DATE,
    wm.VALUE AS high_temperature
FROM closest_stations cs
JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES wm
ON cs.NOAA_WEATHER_STATION_ID = wm.NOAA_WEATHER_STATION_ID
WHERE wm.VARIABLE_NAME = 'Temperature - Max';
"""

# Execute the weather query
cs.execute(weather_query)
weather_data = cs.fetchall()

# Convert the result to a Pandas DataFrame for analysis
df_weather = pd.DataFrame(weather_data, columns=['zip_code', 'date', 'high_temperature'])
print(df_weather.head())


In [ ]:
# Query to create the supplier_zip_code_weather table
create_table_query = """
CREATE OR REPLACE TABLE supplier_zip_code_weather AS
WITH closest_stations AS (
    -- Map ZIP code to the closest weather station
    SELECT zip_code, NOAA_WEATHER_STATION_ID
    FROM FINAL_PROJECT_DB.FINAL_PROJECT_SCHEMA.ZIP_GEO_MAPPING z
    JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_STATION_INDEX s
    ON TRUE
    QUALIFY ROW_NUMBER() OVER (PARTITION BY z.GEOID ORDER BY 
    (6371 * ACOS(
        COS(RADIANS(z.INTPTLAT)) * COS(RADIANS(s.LATITUDE)) *
        COS(RADIANS(s.LONGITUDE) - RADIANS(z.INTPTLONG)) +
        SIN(RADIANS(z.INTPTLAT)) * SIN(RADIANS(s.LATITUDE))
    )) ASC) = 1
)
-- Join with NOAA weather metrics to get high temperatures for each ZIP code
SELECT
    cs.zip_code,
    wm.DATE,
    wm.VALUE AS high_temperature
FROM closest_stations cs
JOIN WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES wm
ON cs.NOAA_WEATHER_STATION_ID = wm.NOAA_WEATHER_STATION_ID
WHERE wm.VARIABLE_NAME = 'Temperature - Max';
"""

# Execute the query to create the table
cs.execute(create_table_query)
print("Table 'supplier_zip_code_weather' created successfully.")


In [ ]:
# Query to check the first 10 rows of the table
verify_query = "SELECT * FROM supplier_zip_code_weather LIMIT 10;"
cs.execute(verify_query)
rows = cs.fetchall()

# Display the results
for row in rows:
    print(row)


# Question 8:

8. **Join** `purchase_orders_and_invoices`, `supplier_case`, and `supplier_zip_code_weather` based on zip codes and the transaction date. Only include transactions that have matching temperature readings.